In [1]:
import os
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold
from skimage import color, io
import cv2
import json
import pandas as pd

In [2]:
# https://medium.com/jdsc-tech-blog/multioutput-cnn-in-pytorch-c5f702d4915f
##############################################################
# DATA LOADING
##############################################################

#########################
# HUMANS IMAGES
#########################


humans_path = 'pattern_ii_dataset/humans/dataset/synthetic_images'

all_humans = []
    
for root, dirs, files in sorted(os.walk(humans_path)):
    
    for name in files:
        all_humans.append(str(root) + "/" + str(name))
        
print("loaded %d files into human database" % len(all_humans))

#########################
# MONSTERS IMAGES
#########################

monsters_path = 'pattern_ii_dataset/monsters/dataset/synthetic_images'

all_monsters = []
    
for root, dirs, files in sorted(os.walk(monsters_path)):
    
    for name in files:
        all_monsters.append(str(root) + "/" + str(name))
        
print("loaded %d files into monster database" % len(all_monsters))

#########################
# HUMANS ANNOTATIONS
#########################

humans_annotations_path = 'pattern_ii_dataset/humans/dataset/annotations'

all_humans_annotations = []
    
for root, dirs, files in sorted(os.walk(humans_annotations_path)):
    
    for name in files:
        all_humans_annotations.append(str(root) + "/" + str(name))
        
print("loaded %d files into all_humans_annotations database" % len(all_humans_annotations))

print(all_humans_annotations[0])

#########################
# MONSTERS ANNOTATIONS
#########################

monsters_annotations_path = 'pattern_ii_dataset/monsters/dataset/annotations'

all_monsters_annotations = []
    
for root, dirs, files in sorted(os.walk(monsters_annotations_path)):
    
    for name in files:
        all_monsters_annotations.append(str(root) + "/" + str(name))
        
print("loaded %d files into all_monsters_annotations database" % len(all_monsters_annotations))

print(all_monsters_annotations[0])


loaded 15992 files into human database
loaded 5992 files into monster database
loaded 6000 files into all_humans_annotations database
pattern_ii_dataset/humans/dataset/annotations\female/subject_mesh_0001_anno.json
loaded 1000 files into all_monsters_annotations database
pattern_ii_dataset/monsters/dataset/annotations\female/subject_mesh_001_anno.json


In [3]:
##############################################################
# DATA SORTING
##############################################################

##############################
# MONSTERS
##############################

monsters_plain = []
monsters_rgb = []
monsters_texture = []

for monster in all_monsters:
    if "rgb" in monster:
        monsters_rgb.append(monster)
    elif "texture" in monster:
        monsters_texture.append(monster)
    else:
        monsters_plain.append(monster)

##############################
# HUMANS
##############################

humans_plain = []
humans_rgb = []
humans_texture = []

for human in all_humans:
    if "rgb" in human:
        humans_rgb.append(human)
    elif "texture" in human:
        humans_texture.append(human)
    else:
        humans_plain.append(human)
        
##############################
# ANNOTATIONS
##############################


monsters_annotations_df = pd.DataFrame() 

for monster_annotation_path in all_monsters_annotations:
    
    with open (monster_annotation_path, 'r') as file:
    
        # load from json
        monster_annotation = json.load(file)
        sub_df = pd.DataFrame.from_dict(monster_annotation['human_dimensions'], orient='index')
        sub_df = sub_df.transpose()
        
        monsters_annotations_df = monsters_annotations_df.append(sub_df, ignore_index=True)


humans_annotations_df = pd.DataFrame() 

for humans_annotation_path in all_humans_annotations:
    
    with open (humans_annotation_path, 'r') as file:
    
        # load from json
        human_annotation = json.load(file)
        sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
        sub_df = sub_df.transpose()
        
        humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
    
    
#print(humans_annotations_df.head())
#print(monsters_annotations_df.head())
#print(humans_rgb[0])    
#print(monsters_rgb[0])    
#print(humans_texture[0])    
#print(monsters_texture[0])    
print(humans_plain[0])    
#print(monsters_plain[0])

pattern_ii_dataset/humans/dataset/synthetic_images\200x200\pose0\female/subject_mesh_0001.png


In [ ]:
#print(humans_annotations_df.head())
print(monsters_annotations_df.columns)

In [9]:
human_image_list = []

for file in humans_plain:
    human_image = color.rgb2gray(io.imread(file))
    # to vector
    human_image = human_image.flatten()
    
    human_image_list.append(human_image)
    
print("finished")

human_image_tensor = torch.tensor(human_image_list)
human_annotation_tensor = torch.tensor(humans_annotations_df.values)
human_annotation_tensor = torch.cat((human_annotation_tensor,human_annotation_tensor),0)


humans_tensor = torch.cat((human_image_tensor,human_annotation_tensor),1)

print("finished")


<ipython-input-9-0f8c133328b1>:4: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  human_image = color.rgb2gray(io.imread(file))


finished
finished


In [ ]:
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 8, kernel_size=5),
      nn.ReLU(),
      nn.MaxPool2d(stride=2, kernel_size=2),
        
      nn.Conv2d(8, 16, kernel_size=5),
      nn.ReLU(),
      nn.MaxPool2d(stride=2, kernel_size=2),
        
      nn.Flatten(),
      nn.Linear(),
        
      nn.ReLU()
    )

# 20 epochs
# mini_batc_size = 100
# Loss = MSE
# learning_rate = 0.01
# momentum = 0.9
# TODO: regressor funktion
# For layers
# 
# https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html
    
nn.
  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
  
    
    
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # First 2D convolutional layer, taking in 1 input channel (image),
      # outputting 32 convolutional features, with a square kernel size of 3
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      # Second 2D convolutional layer, taking in the 32 input layers,
      # outputting 64 convolutional features, with a square kernel size of 3
      self.conv2 = nn.Conv2d(32, 64, 3, 1)

      # Designed to ensure that adjacent pixels are either all 0s or all active
      # with an input probability
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)

      # First fully connected layer
      self.fc1 = nn.Linear(9216, 128)
      # Second fully connected layer that outputs our 10 labels
      self.fc2 = nn.Linear(128, 10)

my_nn = Net()
print(my_nn)

In [ ]:
class MyData(Dataset):
    def __init__(self, train=True, transform=None):
       
        #Leaving only image related  columns
        feature=df.drop(['age', 'gender', 'race'], axis=1)
        #Setting labels
        label_age=df['age']
        label_gender=df['gender']
        label_race=df['race']
        #Splitting the data into train and validation set
        X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test, y_race_train,\
        y_race_test = train_test_split(feature, label_age, label_gender, label_race, test_size=0.2)
        
        if train==True:
            self.x=X_train
            self.age_y=y_age_train
            self.gender_y=y_gender_train
            self.race_y=y_race_train
        else:
            self.x=X_test
            self.age_y=y_age_test
            self.gender_y=y_gender_test
            self.race_y=y_race_test            
        
        #Applying transformation
        self.transform=transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        image=np.array(self.x.iloc[idx, 0:]).astype(float).reshape(137, 236)
        label1=np.array([self.age_y.iloc[idx]]).astype('float')
        label2=np.array([self.gender_y.iloc[idx]]).astype('float')
        label3=np.array([self.race_y.iloc[idx]]).astype('float')
        
        sample={'image': np.uint8(image), 'label_age': label1,\
                'label_gender': label2,\
                'label_race': label3}
        
        #Applying transformation
        if self.transform:
            sample=self.transform(sample)
            
        return sample